In [1]:
import os
import openai
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import numpy as np

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Load Document:

In [10]:
loader = UnstructuredFileLoader('documents/codicedellastrada.pdf')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
texts = text_splitter.split_documents(documents)

Created a chunk of size 1086, which is longer than the specified 1000
Created a chunk of size 1463, which is longer than the specified 1000
Created a chunk of size 1163, which is longer than the specified 1000
Created a chunk of size 1628, which is longer than the specified 1000
Created a chunk of size 1195, which is longer than the specified 1000
Created a chunk of size 1312, which is longer than the specified 1000
Created a chunk of size 1040, which is longer than the specified 1000
Created a chunk of size 1161, which is longer than the specified 1000
Created a chunk of size 1094, which is longer than the specified 1000
Created a chunk of size 1269, which is longer than the specified 1000
Created a chunk of size 1128, which is longer than the specified 1000
Created a chunk of size 1265, which is longer than the specified 1000
Created a chunk of size 1884, which is longer than the specified 1000
Created a chunk of size 1186, which is longer than the specified 1000
Created a chunk of s

In [13]:
texts[150].page_content

'3. Chiunque viola le disposizioni del presente articolo è soggetto alla sanzione amministrativa\n\ndel pagamento di una somma da €159,00 a €639,00.\n\n4.\n\nAlla violazione delle precedenti disposizioni consegue la sanzione amministrativa accessoria dell’obbligo, per l’autore della stessa, del ripristino a sue spese dei luoghi o della rimozione delle opere abusive secondo le norme del capo I, sezione II, del titolo VI.\n\nArticolo 30\n\nFabbricati, muri e opere di sostegno.\n\n1.\n\nI fabbricati ed i muri di qualunque genere fronteggianti le strade devono essere conservati in modo da non compromettere l’incolumità pubblica e da non arrecare danno alle strade ed alle relative pertinenze.\n\n2.'

In [14]:
texts_reduced=texts[30:50]

In [24]:
print(texts_reduced[15].page_content)

10. Le zone di cui ai commi 8 e 9 sono indicate mediante appositi segnali.

9

11. Nell’ambito delle zone di cui ai commi 8 e 9 e delle altre zone di particolare rilevanza urbanistica nelle quali sussistono condizioni ed esigenze analoghe a quelle previste nei medesimi commi, i comuni hanno facoltà di riservare, con ordinanza del sindaco, superfici o spazi di sosta per veicoli privati dei soli residenti nella zona, a titolo gratuito od oneroso.

12. Per le città metropolitane le competenze della giunta e del sindaco previste dal presente rispettivamente dalla giunta metropolitana e dal sindaco

articolo sono esercitate metropolitano.

13. Chiunque non ottemperi ai provvedimenti di sospensione o divieto della circolazione, è soggetto alla sanzione amministrativa del pagamento di una somma da €80,00 a €318,00.


## Embed Text Chunks:

In [19]:
embedding_function = OpenAIEmbeddings()

In [20]:
db = Chroma.from_documents(texts_reduced, embedding_function, persist_directory="knowledge/")
db.persist()

100%|██████████| 1/1 [00:00<00:00,  1.10it/s]


## Run Queries:

In [25]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=db.as_retriever())
qa.run('Esistono zone di sosta per i residenti?')

' Sì, nei commi 8 e 9 viene indicato che i comuni hanno facoltà di riservare, con ordinanza del sindaco, superfici o spazi di sosta per veicoli privati dei soli residenti nella zona, a titolo gratuito od oneroso.'

## Embedding Analysis:

In [14]:
db.get()

{'ids': ['45680570-1b36-11ee-9429-45c8862a16f8'],
 'embeddings': None,
 'documents': ['Hi, my name is john. I live in milan, Italy and I play the piano. I suffer from cronic heart disease. More specifically I have angina pectoris and a couple of stents.'],
 'metadatas': [{'source': 'documents/test_doc.txt'}]}

In [25]:
embeddings=np.asarray(db.get(include=['embeddings'])['embeddings'])
embeddings.shape

(1, 1536)

#Test loading capabilities:

In [2]:
db=Chroma(persist_directory="knowledge/")